# Fama French Factors 

In [56]:
import pandas as pd
import numpy as np
import datetime as dt
import wrds

import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats
from functools import reduce

In [57]:
conn = wrds.Connection()

Enter your WRDS username [lexic]:xli0121
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
Loading library list...
Done


## 1. Compustat

In [104]:
# Variable description 
## at: total assets 
## seq: shareholder equity 
## pstk: preferred stock (capital), total 
## pstkl: preferred stock liquidating value
## pstkrv: preferred stock redemption value
## txditc: deferred taxes and investment tax credit 
## revt: total revenue
## cogs: cost of goods sold 
## xsga: selling, general and admin expense 
## xint: interest and related expense 
comp = conn.raw_sql("""
                    select gvkey, datadate, at, seq,
                    pstk, pstkl, pstkrv, txditc,
                    revt, cogs, xsga, xint
                    from comp.funda
                    where indfmt='INDL' 
                    and datafmt='STD'
                    and popsrc='D'
                    and consol='C'
                    and datadate >= '01/01/1959'
                    """, date_cols=['datadate'])

comp['year']=comp['datadate'].dt.year

### i. preferred stock

In [105]:
comp['ps'] = np.where(comp['pstkrv'].isnull(), comp['pstkl'], comp['pstkrv'])
comp['ps'] = np.where(comp['ps'].isnull(), comp['pstk'], comp['ps'])
comp['ps']= np.where(comp['ps'].isnull(), 0, comp['ps'])
comp['txditc']=comp['txditc'].fillna(0)

### ii. book equity

In [106]:
comp['be'] = comp['seq'] + comp['txditc'] - comp['ps']
comp['be'] = np.where(comp['be']>0, comp['be'], np.nan)

### iii. operating profitability 

In [107]:
comp['op'] = comp['revt'] - comp['cogs'] - comp['xsga'] - comp['xint']
# scaled by book equity
comp['op'] = np.where((comp['op']>0) & (comp['be']>0), 
                      comp['op']/comp['be'], np.nan)
comp['op'] = np.where(comp['op']>0, comp['op'], np.nan)

### iv. \#years in Compustat

In [108]:
comp = comp.sort_values(by=['gvkey','datadate'])
comp['count'] = comp.groupby(['gvkey']).cumcount() # has the same shape with comp
comp = comp[['gvkey','datadate','year','be', 'at', 'op', 'count']]

## 2. CRSP

In [109]:
crsp_m = conn.raw_sql("""
                      select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                      a.ret, a.retx, a.shrout, a.prc
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1959' and '12/31/2018'
                      and b.exchcd between 1 and 3
                      """, date_cols=['date']) 

# change variable format to int
crsp_m[['permco','permno','shrcd','exchcd']] \
    =crsp_m[['permco','permno','shrcd','exchcd']].astype(int)

# Line up date to be end of month
crsp_m['jdate'] = crsp_m['date'] + MonthEnd(0)

### i. delisting return

In [110]:
# add delisting return
dlret = conn.raw_sql("""
                     select permno, dlret, dlstdt 
                     from crsp.msedelist
                     """, date_cols=['dlstdt'])

dlret.permno = dlret.permno.astype(int)
dlret['dlstdt'] = pd.to_datetime(dlret['dlstdt'])
dlret['jdate'] = dlret['dlstdt' ] + MonthEnd(0)

crsp = pd.merge(crsp_m, dlret, how='left',on=['permno','jdate'])
crsp['dlret'] = crsp['dlret'].fillna(0)
crsp['ret'] = crsp['ret'].fillna(0)

# retadj factors in the delisting returns
crsp['retadj'] = (1 + crsp['ret'])*(1 + crsp['dlret']) - 1

In [111]:
crsp[crsp['ret']==0].head(20)

,permno,permco,date,shrcd,exchcd,ret,retx,shrout,prc,jdate,dlret,dlstdt,retadj
3,10001,7953,1991-05-31,11,3,0.0,0.0,1054.0,-9.87500,1991-05-31,0.0,NaT,0.0
30,10001,7953,1995-04-28,11,3,0.0,0.0,2244.0,7.50000,1995-04-30,0.0,NaT,0.0
33,10001,7953,1995-07-31,11,3,0.0,0.0,2254.0,8.25000,1995-07-31,0.0,NaT,0.0
52,10001,7953,1997-02-28,11,3,0.0,0.0,2357.0,8.62500,1997-02-28,0.0,NaT,0.0
54,10001,7953,1997-04-30,11,3,0.0,0.0,2357.0,8.62500,1997-04-30,0.0,NaT,0.0
55,10001,7953,1997-05-30,11,3,0.0,0.0,2357.0,8.62500,1997-05-31,0.0,NaT,0.0
60,10001,7953,1997-10-31,11,3,0.0,0.0,2378.0,8.87500,1997-10-31,0.0,NaT,0.0
61,10000,7952,1986-01-31,10,3,0.0,NaN,3680.0,-4.37500,1986-01-31,0.0,NaT,0.0
74,10000,7952,1987-02-27,10,3,0.0,0.0,3893.0,-0.40625,1987-02-28,0.0,NaT,0.0
78,10001,7953,1986-01-31,11,3,0.0,NaN,985.0,-6.12500,1986-01-31,0.0,NaT,0.0


### ii. market equity

In [112]:
# shrout is in thousands (m)
crsp['me']=crsp['prc'].abs()*crsp['shrout'] 
crsp = crsp.drop(['dlret','dlstdt','prc','shrout'], axis=1)
crsp = crsp.sort_values(by=['jdate','permco','me'])

### iii. market cap adjustment

In [113]:
# sum of me across different permno under the same permco on a given date
# .groupby().sum() change the shape of df 
crsp_summe = crsp.groupby(['jdate','permco'])['me'].sum().reset_index() 

# largest mktcap within a permco/date
crsp_maxme = crsp.groupby(['jdate','permco'])['me'].max().reset_index()

# join by jdate/maxme to find the largest permno
# keep only the largest permno for a single permco
crsp1 = pd.merge(crsp, crsp_maxme, how='inner', on=['jdate','permco','me'])

# drop me column and replace with the sum me
# join with sum of me to get the correct market cap info
# replace the me of the largest permno with the me of this permco
crsp1 = crsp1.drop(['me'], axis=1)
crsp2 = pd.merge(crsp1, crsp_summe, how='inner', on=['jdate','permco'])

# sort by permno and date and also drop duplicates
crsp2 = crsp2.sort_values(by=['permno','jdate']).drop_duplicates()

### iv. December market cap 

- Book euqity: at the last fiscal yearend in calender year t-1
- Market euqity: price $\times$ \#shares at the end of December in calender year t-1


In [114]:
# keep December market cap
crsp2['year'] = crsp2['jdate'].dt.year
crsp2['month'] = crsp2['jdate'].dt.month
# decme = crsp2[(crsp2['month']==12)]
decme = crsp2[(crsp2['month']==12) & (crsp2['me']>0)]
decme = decme[['permno','date','jdate','me','year']].rename(columns={'me':'dec_me'})

# Fama French dates 
# e.g. July 1985 - June 1986 follows the portfolio constructed in June 1985
# 1985 is the investment year 
crsp2['ffdate'] = crsp2['jdate'] + MonthEnd(-6)
crsp2['ffyear'] = crsp2['ffdate'].dt.year
crsp2['ffmonth'] =crsp2['ffdate'].dt.month
crsp2['1+retx']= 1 + crsp2['retx']
crsp2['1+ret']= 1 + crsp2['ret']
crsp2 = crsp2.sort_values(by=['permno','date'])

### v. weights 

| Calender year | Month | me used | Remarks | FF year month|
| ---- | ---- | ---- | ----| ---- |
| 1985 | Jul | me in Jun | Baseline me | 1985-01|
|     | Aug | me in Jun * return in Jul = me in Jul | | 1985-02 |


In [115]:
# cumret by stock
crsp2['cumretx'] = crsp2.groupby(['permno','ffyear'])['1+retx'].cumprod() # same shape 

# lag cumret
crsp2['lcumretx']= crsp2.groupby(['permno'])['cumretx'].shift(1) # same shape

# lag market cap
crsp2['lme'] = crsp2.groupby(['permno'])['me'].shift(1)

# if first permno then use me/(1+retx) to replace the missing value
crsp2['count'] = crsp2.groupby(['permno']).cumcount()
crsp2['lme'] = np.where(crsp2['count']==0, crsp2['me']/crsp2['1+retx'], crsp2['lme'])

# baseline me
mebase = crsp2[crsp2['ffmonth']==1][['permno','ffyear', 'lme']] \
    .rename(columns={'lme':'mebase'})

# merge result back together
# ffmonth = 1, month = 7, lag me is in month 6 when the portfolio is constructed 
crsp3 = pd.merge(crsp2, mebase, how='left', on=['permno','ffyear'])
crsp3['wt'] = np.where(crsp3['ffmonth']==1, crsp3['lme'], 
                     crsp3['mebase']*crsp3['lcumretx'])

# Dec mc in calender year t-1 is used for June in calender year t
# that is ffyear t
decme['year'] = decme['year'] + 1 # actually means lag one period 
decme = decme[['permno','year','dec_me']]

### vi. MOM

In [116]:
# mom is not computed for month t unless a stock has at least 5 months of good returns 
# in t-12 to t-2
crsp3['mom'] = crsp3.groupby(['permno'])['1+ret'] \
    .rolling(11, min_periods=5).apply(np.prod).shift(2).reset_index()['1+ret']

crsp3['mom'] = crsp3['mom'] - 1
crsp3['mom'] = crsp3['mom'] * 100 / 11
crsp3['mom'] = crsp3['mom'].fillna(-999)

### v. information af of June

In [117]:
# calender year t
crsp3_jun = crsp3[crsp3['month']==6]

crsp_jun = pd.merge(crsp3_jun, decme, how='inner', on=['permno','year'])
crsp_jun = crsp_jun[['permno','date', 'jdate', 
                     'shrcd','exchcd','retadj', 
                     'me','wt','cumretx', 'mom',
                     'mebase','lme','dec_me']]
crsp_jun = crsp_jun.sort_values(by=['permno','jdate']).drop_duplicates()

## 3. CCM

In [118]:
# conn = wrds.Connection()

In [119]:
ccm=conn.raw_sql("""
                  select gvkey, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

# fiscal year t-1 be 
ccm1 = pd.merge(comp[['gvkey','datadate','be', 'at', 'op', 'count']], 
                ccm, how='left',on=['gvkey'])

# the yearend of fiscal year t-1 is different for differnt firms 
# year + a numebr is the same as using lag variable 
ccm1['yearend'] = ccm1['datadate'] + YearEnd(0)
ccm1['jdate'] = ccm1['yearend'] + MonthEnd(6)

### i. How does this link table work?

- datadate has to be within the link date bounds

In [120]:
# set link date bounds
ccm2 = ccm1[(ccm1['jdate']>=ccm1['linkdt'])&(ccm1['jdate']<=ccm1['linkenddt'])]
ccm2 = ccm2[['gvkey','permno','datadate','yearend', 'jdate','be', 'at', 'op', 'count']]

# link comp and crsp
# ccm2 be at yearend be of fiscal year t-1 
# crsp_june me is the Dec me it calender year t-1
# me is in thousands 
ccm_jun = pd.merge(crsp_jun, ccm2, how='inner', on=['permno', 'jdate'])
ccm_jun['beme'] = ccm_jun['be']*1000/ccm_jun['dec_me']

In [121]:
# Investment
# assets at yearend of fiscal year t-2 
ccm_jun['lat'] = ccm_jun.groupby('permno')['at'].shift(1)

ccm_jun['inv'] = np.where((ccm_jun['at']>0) & (ccm_jun['lat']>0), 
                          np.log(ccm_jun['at']/ccm_jun['lat']), np.nan)

C:\Users\lexic\Anaconda3\envs\playground\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


### ii. NYSE breakpoints

In [122]:
ccm_jun['op'] = ccm_jun['op'].fillna(-999)
ccm_jun['inv'] = ccm_jun['inv'].fillna(-999)

In [123]:
# select NYSE stocks for bucket breakdown
# exchcd = 1 and positive beme and positive me and shrcd in (10,11) and at least 2 years in comp
nyse = ccm_jun[(ccm_jun['exchcd']==1) & (ccm_jun['beme']>0) & (ccm_jun['me']>0) & \
               (ccm_jun['op']!=-999) & (ccm_jun['inv']!=-999) & (ccm_jun['mom']!=-999) & \
             (ccm_jun['count']>=1) & ((ccm_jun['shrcd']==10) | (ccm_jun['shrcd']==11))]

# size breakdown
nyse_sz=nyse.groupby(['jdate'])['me'].median() \
    .to_frame().reset_index().rename(columns={'me':'sizemedn'})

# beme breakdown
nyse_bm=nyse.groupby(['jdate'])['beme'].describe(percentiles=[0.3, 0.7]).reset_index()
nyse_bm=nyse_bm[['jdate','30%','70%']].rename(columns={'30%':'bm30', '70%':'bm70'})

# op breakdown 
nyse_op=nyse.groupby(['jdate'])['op'].describe(percentiles=[0.3, 0.7]).reset_index()
nyse_op=nyse_op[['jdate','30%','70%']].rename(columns={'30%':'op30', '70%':'op70'})

# inv breakdown 
nyse_inv=nyse.groupby(['jdate'])['inv'].describe(percentiles=[0.3, 0.7]).reset_index()
nyse_inv=nyse_inv[['jdate','30%','70%']].rename(columns={'30%':'inv30', '70%':'inv70'})

# mom breakdown 
nyse_mom=nyse.groupby(['jdate'])['mom'].describe(percentiles=[0.3, 0.7]).reset_index()
nyse_mom=nyse_mom[['jdate','30%','70%']].rename(columns={'30%':'mom30', '70%':'mom70'})

nyse_collection = [nyse_sz, nyse_bm, nyse_op, nyse_inv, nyse_mom]

nyse_breaks = reduce(lambda left, right: 
               pd.merge(left, right, on=['jdate'], how='inner'), 
               nyse_collection)

# join back breakpoints
ccm1_jun = pd.merge(ccm_jun, nyse_breaks, how='left', on=['jdate'])

In [124]:
# function to assign sz and bm bucket
def sz_bucket(row):
    if row['me']==np.nan:
        value=''
    elif row['me']<=row['sizemedn']:
        value='MC1'
    else:
        value='MC2'
    return value

def bm_bucket(row):
    if 0<=row['beme']<=row['bm30']:
        value = 'BM1'
    elif row['beme']<=row['bm70']:
        value='BM2'
    elif row['beme']>row['bm70']:
        value='BM3'
    else:
        value=''
    return value

def op_bucket(row):
    if -998<row['op']<=row['op30']:
        value = 'OP1'
    elif row['op']<=row['op70']:
        value='OP2'
    elif row['op']>row['op70']:
        value='OP3'
    else:
        value=''
    return value

def inv_bucket(row):
    if -998<row['inv']<=row['inv30']:
        value = 'INV1'
    elif row['inv']<=row['inv70']:
        value='INV2'
    elif row['inv']>row['inv70']:
        value='INV3'
    else:
        value=''
    return value

def mom_bucket(row):
    if -998<row['mom']<=row['mom30']:
        value = 'MOM1'
    elif row['mom']<=row['mom70']:
        value='MOM2'
    elif row['mom']>row['mom70']:
        value='MOM3'
    else:
        value=''
    return value

In [125]:
def funcx(row):
    if row['a'] > 2:
        v = "test"
    else:
        v = ''
    return v

df = pd.DataFrame({
    'a': [1, 2, 3, 4],
    'b': [2, 3, 4, 5]
})

In [126]:
df.apply(funcx, axis=1)

0        
1        
2    test
3    test
dtype: object

In [127]:
# assign MC portfolio
ccm1_jun['szport']=np.where((ccm1_jun['beme']>0)&(ccm1_jun['me']>0)& \
                            (ccm1_jun['count']>=1), 
                            ccm1_jun.apply(sz_bucket, axis=1), '')

# assign BM portfolio
ccm1_jun['bmport']=np.where((ccm1_jun['beme']>0)&(ccm1_jun['me']>0) & \
                            (ccm1_jun['count']>=1), 
                            ccm1_jun.apply(bm_bucket, axis=1), '')

# assign OP portfolio
ccm1_jun['opport']=np.where((ccm1_jun['op']>-999)& \
                            (ccm1_jun['count']>=1), 
                            ccm1_jun.apply(op_bucket, axis=1), '')

# assign INV portfolio
ccm1_jun['invport']=np.where((ccm1_jun['inv']>-999)& \
                            (ccm1_jun['count']>=1), 
                            ccm1_jun.apply(inv_bucket, axis=1), '')

# assign mom portfolio
ccm1_jun['momport']=np.where((ccm1_jun['mom']>-999)& \
                            (ccm1_jun['count']>=1), 
                            ccm1_jun.apply(mom_bucket, axis=1), '')

# create positivebmeme and nonmissport variable
ccm1_jun['posbm']=np.where((ccm1_jun['beme']>0)&(ccm1_jun['me']>0)&(ccm1_jun['count']>=1), 
                           1, 0)

ccm1_jun['nonmissport']=np.where((ccm1_jun['bmport']!='') & \
                                 (ccm1_jun['opport']!='') & \
                                 (ccm1_jun['invport']!='') & \
                                 (ccm1_jun['momport']!=''), 1, 0)

In [128]:
ccm1_jun

,permno,date,jdate,shrcd,exchcd,retadj,me,wt,cumretx,mom,...,inv70,mom30,mom70,szport,bmport,opport,invport,momport,posbm,nonmissport
0,10001,1987-06-30,1987-06-30,11,3,0.051429,5.822125e+03,5.602187e+03,0.959184,0.318225,...,0.162263,0.004458,2.366817,,,,,,0,0
1,10001,1988-06-30,1988-06-30,11,3,-0.012039,6.200000e+03,6.379563e+03,1.063830,1.729404,...,0.154856,-1.578383,0.390095,MC1,BM3,,INV1,MOM3,1,0
2,10001,1989-06-30,1989-06-30,11,3,0.017143,7.007000e+03,6.944000e+03,1.120000,1.881898,...,0.155769,0.510446,2.652474,MC1,BM3,,INV1,MOM2,1,0
3,10001,1990-06-29,1990-06-30,11,3,0.014103,1.005225e+04,9.759750e+03,1.392857,4.470169,...,0.136190,-1.593684,0.976058,MC1,BM3,,INV3,MOM3,1,0
4,10001,1991-06-28,1991-06-30,11,3,0.078481,1.126650e+04,1.018112e+04,1.076923,0.654165,...,0.131639,-1.227897,1.244517,MC1,BM2,,INV1,MOM2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281453,93436,2014-06-30,2014-06-30,11,3,0.155412,2.990859e+07,2.522951e+07,2.236028,10.241221,...,0.104293,0.599959,2.740948,MC2,BM1,OP1,INV3,MOM3,1,1
281454,93436,2015-06-30,2015-06-30,11,3,0.069617,3.409612e+07,3.124667e+07,1.117471,0.799835,...,0.095434,-0.577112,1.641608,MC2,BM1,,INV3,MOM2,1,0
281455,93436,2016-06-30,2016-06-30,11,3,-0.049053,3.142062e+07,2.837276e+07,0.791322,-0.363927,...,0.070078,-1.654544,0.417508,MC2,BM1,,INV3,MOM2,1,0
281456,93436,2017-06-30,2017-06-30,11,3,0.060409,6.033933e+07,5.047460e+07,1.703458,3.699406,...,0.076049,0.524636,2.583253,MC2,BM1,OP1,INV3,MOM3,1,1


In [129]:
# store portfolio assignment as of June
june=ccm1_jun[['permno','date', 'jdate', 'beme', 'op', 'inv',
               'bmport','szport', 'opport', 'invport', 'momport',
               'posbm','nonmissport']]
# june['jdate'] = pd.to_datetime(june['jdate'])
june['ffyear']=june['jdate'].dt.year

# merge back with monthly records
crsp3 = crsp3[['date','permno','shrcd','exchcd','retadj', 'me', 'mom',
               'wt','cumretx','ffyear','jdate']]
ccm3=pd.merge(crsp3, 
        june[['permno','ffyear', 'beme', 'op', 'inv',
              'szport','bmport', 'opport', 'invport', 'momport',
              'posbm','nonmissport']], 
              how='left', on=['permno','ffyear'])

# keeping only records that meet the criteria
ccm4=ccm3[(ccm3['wt']>0)& (ccm3['posbm']==1) & (ccm3['nonmissport']==1) & 
          ((ccm3['shrcd']==10) | (ccm3['shrcd']==11))]

C:\Users\lexic\AppData\Local\Temp/ipykernel_6328/629249719.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  june['ffyear']=june['jdate'].dt.year


In [130]:
ccm4

,date,permno,shrcd,exchcd,retadj,me,mom,wt,cumretx,ffyear,...,beme,op,inv,szport,bmport,opport,invport,momport,posbm,nonmissport
977,1964-07-31,10006,10,1,0.073756,2.306810e+05,3.963914,2.148355e+05,1.073756,1964,...,0.778226,0.183140,0.016582,MC2,BM2,OP1,INV1,MOM3,1.0,1.0
978,1964-08-31,10006,10,1,0.012780,2.317865e+05,5.615622,2.306810e+05,1.078902,1964,...,0.778226,0.183140,0.016582,MC2,BM2,OP1,INV1,MOM3,1.0,1.0
979,1964-09-30,10006,10,1,0.074722,2.491060e+05,4.994809,2.317865e+05,1.159520,1964,...,0.778226,0.183140,0.016582,MC2,BM2,OP1,INV1,MOM3,1.0,1.0
980,1964-10-30,10006,10,1,0.029586,2.564760e+05,6.376143,2.491060e+05,1.193825,1964,...,0.778226,0.183140,0.016582,MC2,BM2,OP1,INV1,MOM3,1.0,1.0
981,1964-11-30,10006,10,1,-0.023276,2.483690e+05,6.622164,2.564760e+05,1.156089,1964,...,0.778226,0.183140,0.016582,MC2,BM2,OP1,INV1,MOM3,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3841780,2018-02-28,93436,11,3,-0.031752,5.794969e+07,2.144175,5.912123e+07,0.948702,2017,...,0.137924,0.011752,1.029848,MC2,BM1,OP1,INV3,MOM3,1.0,1.0
3841781,2018-03-29,93436,11,3,-0.224246,4.517557e+07,3.793606,5.724402e+07,0.735959,2017,...,0.137924,0.011752,1.029848,MC2,BM1,OP1,INV3,MOM3,1.0,1.0
3841782,2018-04-30,93436,11,3,0.104347,4.990246e+07,2.115442,4.440725e+07,0.812754,2017,...,0.137924,0.011752,1.029848,MC2,BM1,OP1,INV3,MOM3,1.0,1.0
3841783,2018-05-31,93436,11,3,-0.031201,4.834545e+07,-1.387647,4.904103e+07,0.787395,2017,...,0.137924,0.011752,1.029848,MC2,BM1,OP1,INV3,MOM3,1.0,1.0


## 4. Portfolios

In [131]:
# function to calculate value weighted return
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return np.nan
    
def wavg2(group, var_list, weight):
    d = np.array(group[var_list])
    w = np.array(group[weight])
    w = w / w.sum()
    try:
        return w.T @ d
    except ZeroDivisionError:
        return np.nan

### i. assign the 24 portfolios

In [132]:
avg_list = ['retadj', 'me', 'beme', 'op', 'inv', 'mom']

# mcbm
mcbm = pd.DataFrame(ccm4.groupby(['jdate','szport','bmport']) \
    .apply(wavg2, avg_list,'wt').to_frame().reset_index())
mcbm[avg_list] = pd.DataFrame(mcbm[0].to_list())
mcbm = mcbm.drop(0, axis=1)
mcbm['me'] = np.log(mcbm['me'])
mcbm['port'] = mcbm['szport'] + mcbm['bmport']

# mcop
mcop = pd.DataFrame(ccm4.groupby(['jdate','szport','opport']) \
    .apply(wavg2, avg_list,'wt').to_frame().reset_index())
mcop[avg_list] = pd.DataFrame(mcop[0].to_list())
mcop = mcop.drop(0, axis=1)
mcop['me'] = np.log(mcop['me'])
mcop['port'] = mcop['szport'] + mcop['opport']

# mcinv
mcinv = pd.DataFrame(ccm4.groupby(['jdate','szport','invport']) \
    .apply(wavg2, avg_list,'wt').to_frame().reset_index())
mcinv[avg_list] = pd.DataFrame(mcinv[0].to_list())
mcinv = mcinv.drop(0, axis=1)
mcinv['me'] = np.log(mcinv['me'])
mcinv['port'] = mcinv['szport'] + mcinv['invport']

# mcmom
mcmom = pd.DataFrame(ccm4.groupby(['jdate','szport','momport']) \
    .apply(wavg2, avg_list,'wt').to_frame().reset_index())
mcmom[avg_list] = pd.DataFrame(mcmom[0].to_list())
mcmom = mcmom.drop(0, axis=1)
mcmom['me'] = np.log(mcmom['me'])
mcmom['port'] = mcmom['szport'] + mcmom['momport']


# # firm count
# vwret_n = ccm4.groupby(['jdate','szport','bmport'])['retadj'] \
#     .count().reset_index().rename(columns={'retadj':'n_firms'})
# vwret_n['MCBMport']= vwret_n['szport'] + vwret_n['bmport']

### ii. Cross-sectional factors without MOM

In [338]:
ff4 = pd.concat([mcbm, mcop, mcinv], axis=0).rename(columns={'retadj': 'vwret'}) \
    .sort_values(by=['jdate', 'port'])
ff4 = ff4[(ff4['jdate'] >= '1963-07-31') & \
            ((ff4['jdate'] <= '2018-08-31'))]
ffmonth = ff4['jdate'].unique()

ff4_wide = ff4.pivot(index='jdate', columns='port', 
                      values=['vwret', 'me', 'beme', 'op', 'inv'])

cl = []
for each in ff4_wide.columns:
    cl.append(each[0] + '_' + each[1])

ff4_wide.columns = cl

ff4_scale = ff4_wide.copy()

char_lst = ['me', 'beme', 'op', 'inv']
slkt = [ [i for i in ff4_scale.columns if i.startswith(j)] for j in char_lst ]

for each in ff4_scale.index:
    for char in slkt:
        val = ff4_scale.loc[each, char]
        ff4_scale.loc[each, char] = (val-np.mean(val))/np.std(val)

ff4_scale = ff4_scale.reset_index()

ff4_long = pd.wide_to_long(
    ff4_scale, 
    stubnames=["vwret", "me", "beme", "op", "inv"], 
    i = "jdate", j = "asset",
    sep='_', suffix=r'\w+'
).reset_index().set_index(["asset", "jdate"])

ff4_ret = np.zeros((662, 5))
count = 0
for t in ffmonth:
    df = ff4_long.xs(key=t, level=1) 
    spec = "I(100 * vwret) ~ 1 + me + beme + op + inv"
    res = smf.ols(formula=spec, data=df).fit(cov_type="HC3")
    ff4_ret[count] = res.params
    count += 1
    
FF4XS = pd.DataFrame(ff4_ret, index=ffmonth)
FF4XS.columns = \
    ['Rz', 'Rmc', 'Rbm', 'Rop', 'Rinv' ]
FF4XS.index.name = 'jdate'

In [691]:
FF4XS

,Rz,Rmc,Rbm,Rop,Rinv
jdate,,,,,
1963-07-31,-0.124050,-1.338182,0.714746,1.346018,-0.161645
1963-08-31,6.214243,-0.939927,-0.227098,-0.422729,-0.717606
1963-09-30,-1.201999,2.115829,0.104176,-0.999574,-0.461297
1963-10-31,3.762123,-0.354223,0.835167,1.383927,0.612009
1963-11-30,-0.873550,0.029688,0.954411,0.573965,-0.511174
...,...,...,...,...,...
2018-04-30,0.840421,0.803375,0.803412,-0.537932,-0.022018
2018-05-31,4.026188,-1.457435,-0.089804,0.123393,0.781371
2018-06-30,1.105873,-0.313334,-0.137358,0.011809,-0.121326


In [701]:
tab1a = FF4XS.describe().T[['mean', 'std']]
tab1a['t(mean)'] = tab1a['mean']/np.sqrt(np.square(tab1a['std'])/662)
tab1a

,mean,std,t(mean)
Rz,1.124625,4.960700,5.833026
Rmc,-0.173030,1.580176,-2.817371
Rbm,0.070689,0.900240,2.020330
Rop,0.112079,0.602901,4.783077
Rinv,-0.068000,0.562943,-3.107942


In [704]:
print(tab1a.to_latex())

\begin{tabular}{lrrr}
\toprule
{} &      mean &       std &   t(mean) \\
\midrule
Rz   &  1.124625 &  4.960700 &  5.833026 \\
Rmc  & -0.173030 &  1.580176 & -2.817371 \\
Rbm  &  0.070689 &  0.900240 &  2.020330 \\
Rop  &  0.112079 &  0.602901 &  4.783077 \\
Rinv & -0.068000 &  0.562943 & -3.107942 \\
\bottomrule
\end{tabular}



## iii. Cross-sectional factors with MOM

In [340]:
ff5 = pd.concat([mcbm, mcop, mcinv, mcmom], axis=0).rename(columns={'retadj': 'vwret'}) \
    .sort_values(by=['jdate', 'port'])
ff5 = ff5[(ff5['jdate'] >= '1963-07-31') & \
            ((ff5['jdate'] <= '2018-08-31'))]
ffmonth = ff5['jdate'].unique()

ff5_wide = ff5.pivot(index='jdate', columns='port', 
                      values=['vwret', 'me', 'beme', 'op', 'inv', 'mom'])

cl = []
for each in ff5_wide.columns:
    cl.append(each[0] + '_' + each[1])

ff5_wide.columns = cl

ff5_scale = ff5_wide.copy()

char_lst = ['me', 'beme', 'op', 'inv', 'mom']
slkt = [ [i for i in ff5_scale.columns if i.startswith(j)] for j in char_lst ]

for each in ff5_scale.index:
    for char in slkt:
        val = ff5_scale.loc[each, char]
        ff5_scale.loc[each, char] = (val-np.mean(val))/np.std(val)

ff5_scale = ff5_scale.reset_index()
ff5_long = pd.wide_to_long(
    ff5_scale, 
    stubnames=["vwret", "me", "beme", "op", "inv", 'mom'], 
    i = "jdate", j = "asset",
    sep='_', suffix=r'\w+'
).reset_index().set_index(["asset", "jdate"])

ff5_ret = np.zeros((662, 6))
count = 0
for t in ffmonth:
    df = ff5_long.xs(key=t, level=1) 
    spec = "I(100 * vwret) ~ 1 + me + beme + op + inv + mom"
    res = smf.ols(formula=spec, data=df).fit(cov_type="HC3")
    ff5_ret[count] = res.params
    count += 1
    
FF5XS = pd.DataFrame(ff5_ret, index=ffmonth)
FF5XS.columns = ['Rz', 'Rmc', 'Rbm', 'Rop', "Rinv", 'Rmom']
FF5XS.index.name = 'jdate'

In [692]:
FF5XS

,Rz,Rmc,Rbm,Rop,Rinv,Rmom
jdate,,,,,,
1963-07-31,-0.064652,-1.199689,0.683456,0.981555,0.082240,0.282000
1963-08-31,6.124119,-1.137557,-0.565950,-0.475909,-0.632015,0.953363
1963-09-30,-1.161692,2.316133,0.631745,-0.804672,-0.174529,-0.143914
1963-10-31,3.891496,0.176737,1.312571,1.252177,1.190425,1.501595
1963-11-30,-0.796660,0.551129,0.857449,0.023435,0.129641,1.374585
...,...,...,...,...,...,...
2018-04-30,0.798344,0.771251,0.688236,-0.324010,0.104382,-0.454754
2018-05-31,4.090633,-1.574677,0.472962,0.358837,0.185215,1.582535
2018-06-30,1.129293,-0.425851,0.002434,0.307907,-0.040348,-0.142710


In [702]:
tab1b = FF5XS.describe().T[['mean', 'std']]
tab1b['t(mean)'] = tab1b['mean']/np.sqrt(np.square(tab1b['std'])/662)
tab1b

,mean,std,t(mean)
Rz,1.126587,4.973823,5.827783
Rmc,-0.121804,1.464663,-2.139693
Rbm,0.071362,0.840459,2.184639
Rop,0.080889,0.521735,3.989042
Rinv,-0.056269,0.471419,-3.071078
Rmom,0.108964,0.935353,2.997353


In [705]:
print(tab1b.to_latex())

\begin{tabular}{lrrr}
\toprule
{} &      mean &       std &   t(mean) \\
\midrule
Rz   &  1.126587 &  4.973823 &  5.827783 \\
Rmc  & -0.121804 &  1.464663 & -2.139693 \\
Rbm  &  0.071362 &  0.840459 &  2.184639 \\
Rop  &  0.080889 &  0.521735 &  3.989042 \\
Rinv & -0.056269 &  0.471419 & -3.071078 \\
Rmom &  0.108964 &  0.935353 &  2.997353 \\
\bottomrule
\end{tabular}



### iv. Time-series factor

In [141]:
# time series factor
ts = ff5.pivot(index='jdate', columns='port', values=['vwret'])
cl = []
for i in ts.columns:
    cl.append(i[1])
ts.columns = cl

ts['SMB'] = (ts['MC1BM1'] + ts['MC1BM2'] + ts['MC1BM3'] + ts['MC1OP1'] + ts['MC1OP2'] + ts['MC1OP3'] + ts['MC1INV1'] + ts['MC1INV2'] + ts['MC1INV3'])/9 - \
            (ts['MC2BM1'] + ts['MC2BM2'] + ts['MC2BM3'] + ts['MC2OP1'] + ts['MC2OP2'] + ts['MC2OP3'] + ts['MC2INV1'] + ts['MC2INV2'] + ts['MC2INV3'])/9
ts['SMB'] = ts['SMB'] * 100

ts['HML'] = (ts['MC1BM3'] + ts['MC2BM3'])/2 - (ts['MC1BM1'] + ts['MC2BM1'])/2
ts['HML'] = ts['HML'] * 100

ts['RMW'] = (ts['MC1OP3'] + ts['MC2OP3'])/2 - (ts['MC1OP1'] + ts["MC2OP1"])/2
ts['RMW'] = ts['RMW'] * 100

ts['CMA'] = (ts['MC1INV1'] + ts['MC2INV1'])/2 - (ts['MC1INV3'] + ts["MC2INV3"])/2
ts['CMA'] = ts['CMA'] * 100

ts['UMD'] = (ts['MC1MOM3'] + ts['MC2MOM3'])/2 - (ts['MC1MOM1'] + ts["MC2MOM1"])/2
ts['UMD'] = ts['UMD'] * 100

ts = ts[['SMB', 'HML', 'RMW', 'CMA', 'UMD']]

## 5. Test portfolios

In [145]:
path = "C:\\Users\\lexic\\OneDrive - HKUST Connect\\Workspace\\Compare_XS_and_TS\\data\\"

In [226]:
MCBM = pd.read_excel(path + 'MCBM25.xlsx', header=15, index_col=0)
MCBM.index.name = "jdate"
MCBM = MCBM[444:1106]
MCBM = MCBM.reset_index()
MCBM['jdate'] =  MCBM['jdate'].astype('string')
MCBM['jdate'] =  MCBM['jdate'].apply(lambda x: datetime.strptime(x, "%Y%m")) + MonthEnd(0)

In [227]:
MCOP = pd.read_excel(path + 'MCOP25.xlsx', header=22, index_col=0)
MCOP.index.name = "jdate"
MCOP = MCOP.iloc[:662]
MCOP = MCOP.reset_index()
MCOP['jdate'] =  MCOP['jdate'].astype('string')
MCOP['jdate'] =  MCOP['jdate'].apply(lambda x: datetime.strptime(x, "%Y%m")) + MonthEnd(0)

In [228]:
MCINV = pd.read_excel(path + 'MCINV25.xlsx', header=16, index_col=0)
MCINV.index.name = "jdate"
MCINV = MCINV.iloc[:662]
MCINV = MCINV.reset_index()
MCINV['jdate'] =  MCINV['jdate'].astype('string')
MCINV['jdate'] =  MCINV['jdate'].apply(lambda x: datetime.strptime(x, "%Y%m")) + MonthEnd(0)

In [235]:
MCMOM = pd.read_excel(path + 'MCMOM25.xlsx', header=11, index_col=0)
MCMOM.index.name = "jdate"
MCMOM = MCMOM[438:1100]
MCMOM = MCMOM.reset_index()
MCMOM['jdate'] =  MCMOM['jdate'].astype('string')
MCMOM['jdate'] =  MCMOM['jdate'].apply(lambda x: datetime.strptime(x, "%Y%m")) + MonthEnd(0)

In [238]:
MCBeta = pd.read_excel(path + 'MCBeta25.xlsx', header=16, index_col=0)
MCBeta.index.name = "jdate"
MCBeta = MCBeta.iloc[:662]
MCBeta = MCBeta.reset_index()
MCBeta['jdate'] =  MCBeta['jdate'].astype('string')
MCBeta['jdate'] =  MCBeta['jdate'].apply(lambda x: datetime.strptime(x, "%Y%m")) + MonthEnd(0)

In [240]:
MCAccr = pd.read_excel(path + 'MCAccr25.xlsx', header=18, index_col=0)
MCAccr.index.name = "jdate"
MCAccr = MCAccr.iloc[:662]
MCAccr = MCAccr.reset_index()
MCAccr['jdate'] =  MCAccr['jdate'].astype('string')
MCAccr['jdate'] =  MCAccr['jdate'].apply(lambda x: datetime.strptime(x, "%Y%m")) + MonthEnd(0)

In [243]:
MCVar = pd.read_excel(path + 'MCVar25.xlsx', header=19, index_col=0)
MCVar.index.name = "jdate"
MCVar = MCVar.iloc[:662]
MCVar = MCVar.reset_index()
MCVar['jdate'] =  MCVar['jdate'].astype('string')
MCVar['jdate'] =  MCVar['jdate'].apply(lambda x: datetime.strptime(x, "%Y%m")) + MonthEnd(0)

In [280]:
MCNSI = pd.read_excel(path + 'MCNSI35.xlsx', header=18, index_col=0)
MCNSI.index.name = "jdate"
MCNSI = MCNSI.iloc[:662]
MCNSI = MCNSI.reset_index()
MCNSI['jdate'] =  MCNSI['jdate'].astype('string')
MCNSI['jdate'] =  MCNSI['jdate'].apply(lambda x: datetime.strptime(x, "%Y%m")) + MonthEnd(0)

In [ ]:
ff5 = pd.read_excel(path + 'FF5.xlsx', header=3, index_col=0)
ff5.index.name = "jdate"
ff5 = ff5.iloc[:662]
ff5 = ff5.reset_index()
ff5['jdate'] =  ff5['jdate'].astype('string')
ff5['jdate'] =  ff5['jdate'].apply(lambda x: datetime.strptime(x, "%Y%m")) + MonthEnd(0)

In [681]:
MOMX = [MCBM, MCOP, MCINV, MCBeta, MCAccr, MCVar, MCNSI]
MOMX = reduce(lambda left, right: pd.merge(left, right, on=['jdate'], how='inner'), MOMX)
MOMX = MOMX.set_index(['jdate'])

MOM = [MCBM, MCOP, MCINV, MCMOM, MCBeta, MCAccr, MCVar, MCNSI]
MOM = reduce(lambda left, right: pd.merge(left, right, on=['jdate'], how='inner'), MOM)
MOM = MOM.set_index(['jdate'])

asset185 = list(MOMX.columns)
asset210 = list(MOM.columns)

MOMX = pd.merge(MOMX, ff5[['Mkt-RF', 'RF']], on=['jdate'], how='inner')
MOMX = pd.merge(MOMX, FF4XS, on=['jdate'], how='inner')
MOMX = pd.merge(MOMX, ts, on=['jdate'], how='inner')

MOM = pd.merge(MOM, ff5[['Mkt-RF', 'RF']], on=['jdate'], how='inner')
MOM = pd.merge(MOM, FF5XS, on=['jdate'], how='inner')
MOM = pd.merge(MOM, ts, on=['jdate'], how='inner')

In [653]:
from scipy.stats import f
from numpy.linalg import inv

# GRS test
def GRS(alpha, factor, V):
    
    # alpha: N*1
    # factor: T*K
    # V: N*N
    N = len(alpha)
    T = factor.shape[0]
    K = factor.shape[1]
    
    alpha = np.array(alpha)
    # mu: K*1
    mu = factor.mean(axis=0)
    factor = factor - mu
    sigmaF = (factor.T @ factor)/(T-K)
    
    # test statistic
    J = ((T - N - K )/N) * (1 + mu.T @ inv(sigmaF) @ mu)**(-1)  * (alpha.T @ inv(V) @ alpha)
    
    d1 = N 
    d2 = T - N - K
    p = 2 * f.sf(J, d1, d2) 
    return [J, p]

In [654]:
# generate tables 
def tableGenerator(data, factor, rf, assets, row_name):
    alpha = []
    alpha_t = []
    alpha_se = []
    rbar = []
    R2 = []
    residual = []
    resid_se = []
    for each in assets:
        y = data[each] - data[rf]     
        x = data[factor].astype('float')
        X = sm.add_constant(x)
        res = sm.OLS(y.astype('float'), X.astype('float') ).fit()
        alpha.append(res.params[0])
        alpha_t.append(res.tvalues[0])
        alpha_se.append(res.bse[0])
        rbar.append(np.mean(y))
        R2.append(res.rsquared_adj)
        residual.append(res.resid)
        s = (res.resid.T @ res.resid)/(X.shape[0] - X.shape[1])
        resid_se.append(np.sqrt(s))    
    
    alpha = np.array(alpha)
    residual = np.array(residual)
    V = (residual @ residual.T)/662
    
    output.loc[row_name, :] = [
        np.mean([np.abs(i) for i in alpha]),
        np.mean([np.abs(i) for i in alpha_t]),
        np.mean([i**2 for i in alpha])/np.var(rbar),
        np.mean([(i**2 - j**2) for i, j in zip(alpha, alpha_se)])/np.var(rbar),
        np.mean(R2),
        np.mean(alpha_se),
        np.mean(resid_se),
        np.max(alpha.T @ inv(V) @ alpha),
        GRS(alpha, x, V)[0],
        GRS(alpha, x, V)[1]
    ]

In [686]:
output = pd.DataFrame({
    'A[a]': [], 
    'A[t(a)]': [],
    'Aa2/Vr': [],
    'Alambda2/Vr': [],
    'AR2': [],
    'As(a)': [],
    'As(e)': [],
    'Sh2(a)': [],
    'J': [],
    'p(J)': []
})


In [688]:
tableGenerator(
    data = MOMX,
    factor = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA'],
    rf = 'RF',
    assets = asset185,
    row_name = 'A1a'
)

tableGenerator(
    data = MOMX,
    factor = ['Mkt-RF', 'Rmc', 'Rbm', 'Rop', 'Rinv'],
    rf = 'RF',
    assets = asset185,
    row_name = 'A1b'
)

tableGenerator(
    data = MOMX,
    factor = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'UMD'],
    rf = 'RF',
    assets = asset185,
    row_name = 'A1c'
)

tableGenerator(
    data = MOM,
    factor = ['Mkt-RF', 'Rmc', 'Rbm', 'Rop', 'Rinv', 'Rmom'],
    rf = 'RF',
    assets = asset185,
    row_name = 'A1d'
)

tableGenerator(
    data = MOM,
    factor = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'UMD'],
    rf = 'RF',
    assets = asset210,
    row_name = 'A2a'
)

tableGenerator(
    data = MOM,
    factor = ['Mkt-RF', 'Rmc', 'Rbm', 'Rop', 'Rinv', 'Rmom'],
    rf = 'RF',
    assets = asset210,
    row_name = 'A2b'
)

In [689]:
tableGenerator(
    data = MOMX,
    factor = ['Rmc', 'Rbm', 'Rop', 'Rinv'],
    rf = 'Rz',
    assets = asset185,
    row_name = 'B1a'
)

tableGenerator(
    data = MOM,
    factor = ['Rmc', 'Rbm', 'Rop', 'Rinv', 'Rmom'],
    rf = 'Rz',
    assets = asset185,
    row_name = 'B1b'
)

tableGenerator(
    data = MOM,
    factor = ['Rmc', 'Rbm', 'Rop', 'Rinv', 'Rmom'],
    rf = 'Rz',
    assets = asset210,
    row_name = 'B2'
)

In [706]:
output

,A[a],A[t(a)],Aa2/Vr,Alambda2/Vr,AR2,As(a),As(e),Sh2(a),J,p(J)
A1a,0.108215,1.490390,0.563087,0.429955,0.893397,0.070530,1.746342,1.327730,3.138878,3.599521e-23
A1b,0.112899,1.512703,0.596073,0.454364,0.888461,0.072818,1.788451,1.311580,3.051265,4.404656e-22
A1c,0.105802,1.457900,0.530479,0.396077,0.893783,0.070878,1.743385,1.318147,3.069466,2.740822e-22
A1d,0.132354,1.632447,0.780694,0.609866,0.869705,0.079876,1.957719,1.335062,3.086886,1.666302e-22
A2a,0.117273,1.543805,0.552483,0.436650,0.890484,0.072610,1.786007,1.672920,3.249679,2.138327e-25
A2b,0.139411,1.685155,0.707637,0.565663,0.869104,0.080693,1.977730,1.674363,3.229500,3.838904e-25
B1a,0.085600,1.079115,0.439716,0.290154,0.382691,0.074583,1.860865,1.252631,3.012772,1.268525e-21
B1b,0.089293,1.046778,0.502754,0.333835,0.326260,0.079178,1.966666,1.262085,3.002459,1.782010e-21
B2,0.096518,1.116212,0.468204,0.328049,0.331755,0.079899,1.984584,1.590638,3.157023,2.974923e-24


In [707]:
print(output.to_latex())

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &      A[a] &   A[t(a)] &    Aa2/Vr &  Alambda2/Vr &       AR2 &     As(a) &     As(e) &    Sh2(a) &         J &          p(J) \\
\midrule
A1a &  0.108215 &  1.490390 &  0.563087 &     0.429955 &  0.893397 &  0.070530 &  1.746342 &  1.327730 &  3.138878 &  3.599521e-23 \\
A1b &  0.112899 &  1.512703 &  0.596073 &     0.454364 &  0.888461 &  0.072818 &  1.788451 &  1.311580 &  3.051265 &  4.404656e-22 \\
A1c &  0.105802 &  1.457900 &  0.530479 &     0.396077 &  0.893783 &  0.070878 &  1.743385 &  1.318147 &  3.069466 &  2.740822e-22 \\
A1d &  0.132354 &  1.632447 &  0.780694 &     0.609866 &  0.869705 &  0.079876 &  1.957719 &  1.335062 &  3.086886 &  1.666302e-22 \\
A2a &  0.117273 &  1.543805 &  0.552483 &     0.436650 &  0.890484 &  0.072610 &  1.786007 &  1.672920 &  3.249679 &  2.138327e-25 \\
A2b &  0.139411 &  1.685155 &  0.707637 &     0.565663 &  0.869104 &  0.080693 &  1.977730 &  1.674363 &  3.229500 &  3.838904e-25 \\
B1a &  0.085600 